## Detailed Article Explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/542136/retrieval-augmented-generation-with-claude-3-5-sonnet

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Installing and Importing Required Libraries

In [ ]:
!pip install -U langchain
!pip install -U langchain-anthropic
!pip install langchain-openai
!pip install pypdf
!pip install faiss-cpu

In [1]:
from langchain_anthropic import ChatAnthropic

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
import os


## Generating Default Response with Claude 3.5 Sonnet

In [5]:
anthropic_api_key = os.environ.get('ANTHROPIC_API_KEY')

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620",
                     anthropic_api_key = anthropic_api_key,
                     temperature = 0.3)

result = llm.invoke("Write a funny poem for an ice cream shop on a beach.")
print(result.content)

Here's a funny poem for an ice cream shop on a beach:

Sandy Scoops and Beachy Treats

Come one, come all, to our beachside parlor,
Where ice cream's the star, and fun's the caller!
Our cones are crispy, our flavors are wacky,
(Just don't drop your scoop, or things might get tacky!)

Try our Sunburn Strawberry, it's sure to cool,
Or Coconut Castaway, it's simply rule!
Shark Bite Chocolate will give you a thrill,
While Seagull Surprise might fit the bill!

We've got sprinkles that sparkle like ocean spray,
And toppings so wild, they'll make your day!
But watch out for the crabs, they love our cream,
They'll pinch at your toes and make you scream!

So grab a spoon, or better yet, two,
And dive into flavors both frosty and new.
At Sandy Scoops, we promise delight,
Just keep your ice cream away from the gulls in flight!


In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", '{assistant}'),
    ("user", "{input}")
])

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

result = chain.invoke(
    {"assistant": "You are a comedian",
     "input": "Write a funny poem for a music store on a beach."}
)
print(result)

Here's a funny poem for a music store on a beach:

Sandy Tunes and Salty Strings

Our beach-side shop's a musical treat,
Where waves and melodies sweetly meet.
We've got ukuleles for your luau nights,
And waterproof headphones for aquatic delights.

Our guitars are coated in sunscreen, you see,
To protect them from that harsh UV.
Our drums are filled with shells and sand,
Perfect for your beachfront band.

Our sheet music's printed on inflatable rafts,
So your tunes can float with summer's drafts.
Our microphones are shaped like ice cream cones,
For those cool vocals and smooth baritones.

So come on in, don't be a beach bum,
We've got instruments to make your vacation hum.
Just watch out for the singing crabs in aisle three,
They're our in-house band – admission's free!



## RAG with Claude 3.5 Sonnet

### Step 1: Loading and Splitting Documents

In [8]:
loader = PyPDFLoader("https://web.archive.org/web/20170809122528id_/http://global-settlement.org/pub/The%20English%20Constitution%20-%20by%20Walter%20Bagehot.pdf")
docs = loader.load_and_split()

### Step 2: Creating Embeddings

In [10]:
openai_key = os.environ.get('OPENAI_API_KEY')

embeddings = OpenAIEmbeddings(openai_api_key = openai_key)

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

### Step 3: Crafting the Prompt Template

In [11]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

Question: {input}

Context: {context}
"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

### Step 4: Setting Up the Retriever

In [12]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

### Step 5: Generating Responses

In [13]:
def generate_response(query):
    response = retrieval_chain.invoke({"input": query})
    print(response["answer"])

In [15]:
query = "What is the total number of members of the house of commons?"
generate_response(query)

Based on the provided context, the total number of members of the House of Commons is 658. This can be seen in the following excerpt:

"Here are 658 persons, collected from all parts of England, different in nature, different in interests, different in look, and language."


In [18]:
query = "What is the difference between the house of lords and house of commons? How members are elected for both?"
generate_response(query)

Based on the provided context, here are the key differences between the House of Lords and House of Commons:

1. Composition:
- House of Lords: Consists of peers (aristocracy) who are not elected.
- House of Commons: Consists of elected members representing constituencies.

2. Power and influence:
- House of Lords: Has less power and influence, mainly acts as a revising chamber.
- House of Commons: Has more power and is the main legislative body.

3. Accountability:
- House of Lords: Not accountable to constituents, more independent.
- House of Commons: Accountable to constituents, influenced by public opinion.

4. Activity level:
- House of Lords: Described as less active, with fewer members regularly participating.
- House of Commons: More active, with higher attendance and participation.

5. Social influence:
- House of Lords: Seen as "corruptors" who give social bribes.
- House of Commons: Some members influenced by social bribes from aristocracy.

Regarding how members are elected

In [19]:
query = "How many players participate in a football game?"
generate_response(query)

I apologize, but I cannot answer the question "How many players participate in a football game?" based solely on the provided context. The given text does not contain any information about football or the number of players in a football game. The context appears to be discussing political systems, voting, and parliamentary representation. It does not mention anything related to football or sports in general.
